# Підготовка даних

## Файл csv із збігами та їх прізвищами

Мають бути такі колонки

- surname - прізвище предка збіженця. У чоловічому роді, українською мовою
- name - ім'я збіженця
- cm - число, спільна днк в сантиморганах
- url - покликання на збіженця

# Параметри

In [41]:
# Всі необхідні файли потрібно покласи у цей каталог
directory = "data"

# Файл із збіженцями та прізвищами їх предків
filename = "DNA Genealogy - Surnames_ancestry_bio"

multy = 1000

# Відкриваємо вхідний файл із збіженцями прізвищами їх предків та базу даних

In [42]:
import sqlite3

db = sqlite3.connect(f'{directory}/surnames.sqlite3')

In [43]:
import pandas as pd
matches_df = pd.read_csv(f'{directory}/{filename}.csv')
matches_df

,surname,name,cm,url
0,Влодарчик,N.W.,20,https://www.ancestry.com/discoveryui-matches/c...
1,Корба,N.W.,20,https://www.ancestry.com/discoveryui-matches/c...
2,Бандурик,Bruce Chymboryk,19,https://www.ancestry.com/discoveryui-matches/c...
3,Грамяк,Bruce Chymboryk,19,https://www.ancestry.com/discoveryui-matches/c...
4,Бондарчук,Bruce Chymboryk,19,https://www.ancestry.com/discoveryui-matches/c...
5,Котвиця,N.K.,19,https://www.ancestry.com/discoveryui-matches/c...
6,Муравяк,Zuzanna Murawiak,19,https://www.ancestry.com/discoveryui-matches/c...
7,Мазгай,M.D.,19,https://www.ancestry.com/discoveryui-matches/c...
8,Білий,Nicole Doyle,15,https://www.ancestry.com/discoveryui-matches/c...
9,Рапко,SVL55,15,https://www.ancestry.com/discoveryui-matches/c...


# Підготовка таблиць

In [44]:
table_distribution = "distribution"
table_raw = "raw"

cursor = db.cursor()


In [45]:
cursor.execute(f'DROP TABLE IF EXISTS {table_distribution}')
cursor.execute(
    f'''
    CREATE TABLE {table_distribution} (
    	"region"	INTEGER NOT NULL,
    	"county"	INTEGER NOT NULL,
    	"city"	TEXT NOT NULL,
    	"score"	REAL NOT NULL,
    	PRIMARY KEY("region","county","city")
    )
    '''
)

In [46]:
cursor.execute(f'DROP TABLE IF EXISTS {table_raw}')
cursor.execute(
    f'''
    CREATE TABLE "{table_raw}" (
    	"surname"	INTEGER NOT NULL,
    	"city"	TEXT NOT NULL,
    	"county"	INTEGER NOT NULL,
    	"region"	INTEGER NOT NULL,
    	"cnt"	INTEGER NOT NULL,
    	"match"	INTEGER NOT NULL,
    	"url"	TEXT NOT NULL
    );
    '''
)

In [47]:
db.commit()

# Групування збіженців

In [48]:
match_grouped = matches_df.groupby(["url", "cm", "name"]).surname

In [49]:
for (url, cm, match), surnames in match_grouped:
    print(match)
    cm_float = float(cm)
    str_surnames = "','".join(map(lambda t: t.upper().replace("'", "''"), surnames))
    select = f'''
        SELECT 
            COUNT(*) AS cn,
        	"Областьнародження" as region,
        	IFNULL("Районнародження",0) as county,
        	"Названаселеногопунктународження(Україна)" as city,
            "Прізвищє" as surname
        FROM people 
        where 
        	"Прізвищє" in ('{str_surnames}') 
        	AND "Стать"=1 
        	AND "Названаселеногопунктународження(Україна)" IS NOT NULL 
        	AND "Областьнародження" IS NOT NULL
        GROUP BY region,
        	county,
        	city,
            surname
    '''

    def rec(row):
        (cnt, region, county, city, surname) = row
        return {
            'surname': surname,
            "region": region,
            "county": county,
            "city": city,
            "cnt": cnt
        }

    cursor.execute(select)
    res = list(map(rec, cursor.fetchall()))
    total = 0
    for t in res:
        # print(t)
        total = total + t["cnt"]

    def update_params(r):
        score = (cm_float*r["cnt"]*multy)/total
        return (r["region"], r["county"], r["city"], score, score)

    def insert_raw_params(r):
        return (r["surname"], r["city"], r["county"], r["region"], r["cnt"], match, url)

    cursor.executemany(
        f'''
        INSERT INTO {table_distribution}(region, county, city, score)
          VALUES(?, ?, ?, ?)
          ON CONFLICT(region, county, city) DO UPDATE SET
            score=score+?;
        ''',
        list(map(update_params, res))
    )

    cursor.executemany(
        f'''
        INSERT INTO {table_raw}(surname, city, county, region, cnt, match, url)
          VALUES(?, ?, ?, ?, ?, ?, ?)
        ''',
        list(map(insert_raw_params, res))
    )
    
    db.commit()
    # break

print("All records processed")
    

gailen160
Susan Damico
jenniferheebner14
Leslie Svoboda
Gerald Lapinsky
Zuzanna Murawiak
George_pawlush
N.K.
Stephanie Wupper
sooci187
Tim Ulewicz
Bohdan Borys
Inga Carlson
Gary Gray
SVL55
Diana haladay
blt514
ts23177
N.W.
Kinga Barrow
Bruce Chymboryk
Nicole Doyle
Bernard Syty
Deborah Santilli
M.D.
Olha Hrytsaniuk
Timothy Wata
kathleen513
Michael Litwenak
James Cairns
Justin Kralik
Stephanie Paladino
Robert Michalowski
pundits2012
W.F.
Laura Bell
All records processed


# Створимо результуючий документ

In [50]:
import xlsxwriter

In [51]:
import os, errno

resfile = f'{directory}/{filename}.xlsx'
try:
    os.remove(filename)
except OSError:
    pass
workbook = xlsxwriter.Workbook(resfile)

In [52]:
def add_row(ws, rn, content, format = None):
    for i, c in enumerate(content):
        # print(f"Write {c} to ({i},{rn})")
        ws.write(rn, i, c, format)

In [53]:
header_format = workbook.add_format()
header_format.set_bold()

def add_sheet_sql(name, header, sql, width = None):
    worksheet = workbook.add_worksheet(name)
    # if width is not None:
    #     for w in width:
            
    add_row(worksheet, 0, header, header_format)
    cursor.execute(sql)
    for i, row in enumerate(cursor.fetchall()):
        add_row(worksheet, i+1, row)

    worksheet.autofit()

In [54]:
add_sheet_sql("Топ 100", ["Область", "Район", "НП", "Ранг"], 
              '''
            SELECT
            	IFNULL(r.name, d.region) AS region,
            	IFNULL(c.name, d.county) as county,
            	city,
            	d.score
            FROM	
            	(SELECT county, sum(score) AS score
            	FROM vw_distribution
            	GROUP BY county) g
            INNER JOIN vw_distribution d ON g.county = d.county
            INNER JOIN regions r ON d.region=r.id
            LEFT JOIN counties c ON d.county=c.id
            ORDER BY g.score DESC, d.score DESC          
              '''
             )

In [55]:
add_sheet_sql("Райони", ["Область", "Район", "Ранг"], 
              '''
            SELECT
                IFNULL(r.name, d.region) as region,
                IFNULL(c.name, d.county) as county,
                d.score
            FROM
            (SELECT county, region, sum(score) as score
            FROM distribution d
            WHERE county > 0
            GROUP BY county, region
            ORDER BY sum(score) DESC) as d
            LEFT JOIN counties as c ON d.county = c.id
            LEFT JOIN regions as r on d.region = r.id   
              '''
             )

In [56]:
add_sheet_sql("Області", ["Область", "Ранг"], 
              '''
            SELECT
                IFNULL(r.name, d.region) as region,
                d.score
            FROM
            (SELECT region, sum(score) as score
            FROM distribution d
            GROUP BY region
            ORDER BY sum(score) DESC) as d
            LEFT JOIN regions as r on d.region = r.id
              '''
             )

In [57]:
add_sheet_sql("Всі Села", ["Область", "Район", "НП", "Ранг"], 
              '''
            SELECT
                IFNULL(r.name, d.region) AS region,
                IFNULL(c.name, d.county) as county,
                city,
                score
            FROM distribution d
            LEFT JOIN regions r ON d.region=r.id
            LEFT JOIN counties c ON d.county=c.id
            WHERE county > 0
            ORDER by score DESC        
              '''
             )

In [58]:
add_sheet_sql("Прізвища", ["Прізвище", "Область", "Район", "НП", "Кількість", "Збіг"], 
              '''
            SELECT 
                surname,
                IFNULL(r.name, d.region) as region,
                IFNULL(c.name, d.county) as county,
                d.city,
                cnt,
                match
            FROM raw d
            LEFT JOIN counties as c ON d.county = c.id
            LEFT JOIN regions as r on d.region = r.id
            order by d.surname    
              '''
             )

In [59]:
workbook.close()

# Закрити базу даних

In [60]:
db.close()